# Capstone Project (Mx Data)
## 1. Setup and Reading Files

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import calendar

In [2]:
df = pd.read_csv('mx_raw.csv')
df

,Asset ID,Asset Description,Work ID,Status,Event ID,Event Name,Actual Duration (HRS),date,mx type,mx time,Priority Number
0,LCB3024,Large Canister and Basket Set,MNT-182592,Closed,ME-002747,Semi-Annual Maintenance,0.50,2/28/2018,ME,Semi-Annual,2
1,AB12,Scale,CAL-029177,Closed,CE-002488,Annual Calibration,0.50,9/6/2019,CE,Annual,4
2,AB19,Scale,CAL-024551,Closed,CE-003125,Annual Calibration,0.50,1/31/2018,CE,Annual,4
3,AB2,Scale,CAL-027929,Closed,CE-003394,Annual Calibration,0.50,3/26/2019,CE,Annual,4
4,AB20,Scale,CAL-032226,Closed,CE-002891,Annual Calibration,1.50,11/14/2020,CE,Annual,3
...,...,...,...,...,...,...,...,...,...,...,...
3631,WMC96,WMT CLAMP BASE,CAL-024288,Closed,CE-002450,Quarterly Maintenance,0.15,1/6/2018,CE,Quarterly,2
3632,WMC97,WMT CLAMP BASE,CAL-032345,Closed,CE-002127,Quarterly Calibration,0.50,12/11/2020,CE,Quarterly,1
3633,WMC98,Wedge Clamp,CAL-027282,Closed,CE-002429,Quarterly Calibration,0.10,1/5/2019,CE,Quarterly,2
3634,WMC98,Wedge Clamp,CAL-024291,Closed,CE-002429,Quarterly Maintenance,0.15,1/6/2018,CE,Quarterly,2


## 2. Find Upcoming Events for Subsequent Month

In [3]:
# convert date column to datetime type
df.date = pd.to_datetime(df.date)

In [4]:
# find unique mx times
df['mx time'].unique()

array(['Semi-Annual', 'Annual', 'Quarterly', 'Monthly', 'Daily',
       'Bi-Weekly', '2 Week', '2yr', 'Weekly'], dtype=object)

In [5]:
# create the new date column by add the date and mx time columns
# convert mx time to hours
# create a list of our conditions
conditions = [
    (df['mx time'] == '2yr'),
    (df['mx time'] == 'Annual'),
    (df['mx time'] == 'Semi-Annual'),
    (df['mx time'] == 'Quarterly'),
    (df['mx time'] == 'Monthly'),
    (df['mx time'] == '2 Week'),
    (df['mx time'] == 'Weekly'),
    (df['mx time'] == 'Bi-Weekly'),
    (df['mx time'] == 'Daily')
    ]

# create a list of the values we want to assign for each condition
values = [17520,
          8760,
          4380,
          2190,
          730,
          336,
          168,
          84,
          24
         ]
# create new date column
df['mx_due_date'] = df.date + pd.to_timedelta(np.select(conditions, values), unit='h')
# df['hrs_in_month'] = df.new_date.dt.daysinmonth * 24
df

,Asset ID,Asset Description,Work ID,Status,Event ID,Event Name,Actual Duration (HRS),date,mx type,mx time,Priority Number,mx_due_date
0,LCB3024,Large Canister and Basket Set,MNT-182592,Closed,ME-002747,Semi-Annual Maintenance,0.50,2018-02-28,ME,Semi-Annual,2,2018-08-29 12:00:00
1,AB12,Scale,CAL-029177,Closed,CE-002488,Annual Calibration,0.50,2019-09-06,CE,Annual,4,2020-09-05 00:00:00
2,AB19,Scale,CAL-024551,Closed,CE-003125,Annual Calibration,0.50,2018-01-31,CE,Annual,4,2019-01-31 00:00:00
3,AB2,Scale,CAL-027929,Closed,CE-003394,Annual Calibration,0.50,2019-03-26,CE,Annual,4,2020-03-25 00:00:00
4,AB20,Scale,CAL-032226,Closed,CE-002891,Annual Calibration,1.50,2020-11-14,CE,Annual,3,2021-11-14 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
3631,WMC96,WMT CLAMP BASE,CAL-024288,Closed,CE-002450,Quarterly Maintenance,0.15,2018-01-06,CE,Quarterly,2,2018-04-07 06:00:00
3632,WMC97,WMT CLAMP BASE,CAL-032345,Closed,CE-002127,Quarterly Calibration,0.50,2020-12-11,CE,Quarterly,1,2021-03-12 06:00:00
3633,WMC98,Wedge Clamp,CAL-027282,Closed,CE-002429,Quarterly Calibration,0.10,2019-01-05,CE,Quarterly,2,2019-04-06 06:00:00
3634,WMC98,Wedge Clamp,CAL-024291,Closed,CE-002429,Quarterly Maintenance,0.15,2018-01-06,CE,Quarterly,2,2018-04-07 06:00:00


In [6]:
# find today 
today = dt.date.today()

# slice all mx due next month or past due
next_month_mx = df.loc[(df.new_date.dt.month == today.month + 1) | (df.new_date < pd.to_datetime(today))]

# find sum of avg duration
tot_hrs = sum(next_month_mx['Actual Duration (HRS)'])

# display next month mx
next_month_mx.loc[:,['Asset ID', 'Asset Description', 'Work ID', 'Event Name', 'mx_due_date', 'Priority Number']]

AttributeError: 'DataFrame' object has no attribute 'new_date'

## 3. Calculate total MX hours available

In [14]:
# pd.to_datetime(today).weekday()
next_month = today.month + 1
year = today.year

weekday_count = 0
weekend_count = 0
cal = calendar.Calendar()

# iterate through weeks in month
for week in cal.monthdayscalendar(year, next_month):
    for i, day in enumerate(week):
        # check if in month and weekend
        if day != 0 and i >= 5:
            weekend_count += 1
        # check if in month and weekday
        elif day!= 0 and i < 5:  
            weekday_count += 1



22
8


## 4. Can the MX Crew Finish All Events?
#### 3.1 Yes: Write Events to a CSV
#### 3.2 No: Pick Events by Priority and Write Priority Events to a CSV